In [3]:
import numpy  as np
class Variable:
    def __init__(self,data):
        self.data = data

class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self,x):
        raise NotImplementedError()
    #抛出异常，告诉用户square等函数都是
    #集成于Function，需要自己实现

In [4]:
class Square(Function):
    def forward(self, x):
        return x**2
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)